# 17기 KNN 정규세션 과제

## KNN 구현해보기
### 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.
### 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.
### 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


In [3]:

import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsRegressor, DistanceMetric, NearestNeighbors

from collections import Counter

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [4]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


In [5]:
df.isnull().sum()

User_ID                          0
Product_ID                       0
Gender                           0
Age                              0
Occupation                       0
City_Category                    0
Stay_In_Current_City_Years       0
Marital_Status                   0
Product_Category_1               0
Product_Category_2            1533
Product_Category_3            3454
Purchase                         0
dtype: int64

In [7]:
num_of_null = df.isnull().sum()
percent = (num_of_null / df.isnull().count() * 100)
pd.concat([num_of_null, percent], axis = 1, keys=['# of null', 'Percent']).sort_values(by='Percent', ascending=False)

,# of null,Percent
Product_Category_3,3454,69.107643
Product_Category_2,1533,30.672269
User_ID,0,0.000000
Product_ID,0,0.000000
Gender,0,0.000000
Age,0,0.000000
Occupation,0,0.000000
City_Category,0,0.000000
Stay_In_Current_City_Years,0,0.000000
Marital_Status,0,0.000000


Product_Category_3 변수의 경우에는 약 70%가 null값으로 구성되어있어 제거

In [8]:
df = df.drop("Product_Category_3", axis=1)

In [9]:
# Product_Category_2 변수 살펴보기
df['Product_Category_2'].value_counts()

8.0     588
14.0    496
2.0     448
16.0    392
15.0    345
5.0     258
4.0     246
6.0     148
17.0    132
11.0    110
13.0    102
9.0      65
12.0     40
3.0      35
10.0     30
18.0     23
7.0       7
Name: Product_Category_2, dtype: int64

In [10]:
df['Product_Category_2'].median()

9.0

In [11]:
df['Product_Category_2'].mean()

9.773737373737374

In [12]:
df['Product_Category_2'] = df['Product_Category_2'].fillna(9.)

In [13]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          4998 non-null   float64
 10  Purchase                    4998 non-null   int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 468.6+ KB


In [14]:
df.isnull().sum() 

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Purchase                      0
dtype: int64

 Occupation, Marital_Status, Product_Category_1, Product_Category_2는 범주화(Object)

In [17]:
obj_columns = ['Occupation', 'Marital_Status', 'Product_Category_1', 'Product_Category_2']

for col in obj_columns:
    df[col] = df[col].astype(str)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   User_ID                     4998 non-null   int64 
 1   Product_ID                  4998 non-null   object
 2   Gender                      4998 non-null   object
 3   Age                         4998 non-null   object
 4   Occupation                  4998 non-null   object
 5   City_Category               4998 non-null   object
 6   Stay_In_Current_City_Years  4998 non-null   object
 7   Marital_Status              4998 non-null   object
 8   Product_Category_1          4998 non-null   object
 9   Product_Category_2          4998 non-null   object
 10  Purchase                    4998 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 468.6+ KB


In [20]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,9.0,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,5780


In [22]:
# User_ID는 분석에 필요한 데이터가 아니므로 삭제
df= df.drop('User_ID', axis = 1)

In [23]:
# Feature, Label 분리
y = df['Gender']
X = df.drop("Gender", axis = 1)

In [24]:
# 타깃변수 Label Encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(y)

In [54]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [25]:
# 데이터셋 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                   shuffle = True,
                                                   random_state = 1015,
                                                   stratify = y)

X_train.reset_index(drop = True, inplace=True)
X_test.reset_index(drop = True, inplace=True)

In [31]:
X_train

,Product_ID,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,P00117842,26-35,4,B,1,0,5,13.0,6985
1,P00367942,55+,13,C,1,1,5,9.0,8845
2,P00053842,18-25,4,A,0,0,4,5.0,2772
3,P00253442,26-35,20,A,3,1,5,8.0,8807
4,P00010742,18-25,7,C,1,0,1,8.0,19554
...,...,...,...,...,...,...,...,...,...
3993,P00075342,18-25,4,A,1,0,1,2.0,4070
3994,P00265242,26-35,15,A,2,1,5,8.0,8626
3995,P00197342,26-35,12,A,0,1,1,2.0,7776
3996,P00106742,26-35,0,B,3,0,3,5.0,13223


In [53]:
 y_train

array([1, 1, 1, ..., 0, 1, 1])

In [32]:

cat_columns = [c for (c, t) in zip(X.dtypes.index, X.dtypes) if t == 'O'] 
num_columns = [c for c in X.columns if c not in cat_columns]

In [33]:
cat_columns

['Product_ID',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2']

In [34]:
num_columns

['Purchase']

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

for col in num_columns:
    X_train.loc[:, col] = scaler.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test.loc[:, col] = scaler.transform(np.array(X_test[col]).reshape(-1, 1))

In [36]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse = False)

X_all = pd.concat([X_train, X_test], axis = 0).reset_index(drop=True)

ohe.fit(X_all[cat_columns])

OneHotEncoder(sparse=False)

In [37]:

new_X = ohe.transform(X_all[cat_columns])

ohe_columns = ohe.categories_[0].tolist()

for idx in range(len(ohe.categories_)-1):
    ohe_columns += ohe.categories_[idx+1].tolist()
    
len(ohe_columns)

1872

In [38]:

X_cat = pd.DataFrame(new_X, columns = ohe_columns)
X_cat.head()

,P00000142,P00000242,P00000342,P00000442,P00000642,P00000742,P00001042,P00001142,P00001242,P00001342,...,17.0,18.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [39]:
X_all.drop(columns=cat_columns, inplace=True)

X = pd.concat([X_all, X_cat], axis = 1)

In [46]:
X

,Purchase,P00000142,P00000242,P00000342,P00000442,P00000642,P00000742,P00001042,P00001142,P00001242,...,17.0,18.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,-0.455587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.083118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-1.299251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.090727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.061387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,-0.424748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4994,-1.389765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4995,0.154182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4996,0.161391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# train, test 데이터 분리
X_train = X.iloc[:X_train.shape[0], :]
X_test = X.iloc[X_train.shape[0]:, :]

In [52]:
X_train.shape, X_test.shape,  y_train.shape

((3998, 1873), (1000, 1873), (3748,))

In [50]:
X_test.shape ,y_test.shape

((1000, 1873), (1250,))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print("Test Score : {:.3f}".format(knn.score(X_test, y_test)))


In [ ]:
results = knn.predict(X_test)
results